<a href="https://colab.research.google.com/github/HAITAME/LLM_agents/blob/main/LLM_Workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation

In [2]:
!pip install -q crewai
!pip install -U duckduckgo-search
!pip install google googlesearch-python
!pip install -q 'crewai[tools]'
!pip install langchain_google_genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 774.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

 ### Librairies

In [3]:
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool , WebsiteSearchTool , tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import DuckDuckGoSearchRun
from google.colab import userdata
from googlesearch import search
from bs4 import BeautifulSoup
import requests
import os

### Set gemini pro as llm

In [5]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",
                             verbose = True,
                             temperature = 0.5,
                             google_api_key=userdata.get('gemini_api'))

### Google search

In [6]:
@tool("GoogleSearchTool")
def GoogleSearchTool(question: str) -> str:
    """ Google Search Useful for when you need to  search something one the web"""


    results = []
    for url in search(question, num_results=5):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        paragraphs = soup.find_all('p')
        result_text = ""
        for paragraph in paragraphs:
            result_text += paragraph.text + "\n"
        result_text += f"URL: {url}\n"  # Ajout de la URL à la fin de chaque résultat
        results.append(result_text)
    return "\n".join(results)


### DuckDuckGo search

In [14]:
@tool("DuckDuckGo_Search")
def DuckDuckGo_Search(question: str) -> str:
    """
    DuckDuckGoSearchRun tool

    This tool executes a search query using DuckDuckGo, a search engine known for its privacy features and instant answers.
    It takes a question as input and returns relevant search results from DuckDuckGo.

    Parameters:
    - question (str): The question or query to search for.

    Returns:
    - str: The search results obtained from DuckDuckGo in text format.
    """
    # Initialize the DuckDuckGo search engine
    search = DuckDuckGoSearchRun()

    # Execute the search query and return the results
    return search(question)


### Agents

#### Agent 1  `researcher`

In [34]:
researcher = Agent(
    role="Researcher",
    goal="Generate and searche about video game stories",
    backstory="""The researcher is dedicated to finding valuable information to enrich the game story.
                  With a vast knowledge of video game history, the researcher navigates the depths of
                  the internet to uncover intriguing concepts and ideas that will engage the players in a
                   captivating gaming experience.""",
    verbose=True,
    allow_delegation=False,
    #tools=[GoogleSearchTool],
    llm=llm,
)

#### Agent 2  `writer`

In [35]:
writer = Agent(
    role="Writer",
    goal="Write a story presenting mini-challenges",
    backstory="""The writer is an expert in storytelling, with a keen sense of narrative structure and character development.
                   With a background in creative writing, the writer crafts compelling and immersive game stories that captivate the players.
                    With a passion for gaming, the writer understands the nuances of video game storytelling and brings the game world to life
                    through vivid and engaging narratives.""",
    verbose=True,
    allow_delegation=True,
    llm=llm,
)

### Development Workflow

In [36]:
# Task for the characters
task1 = Task(
    description="""Create unique and interesting characters for the game, each with their own strengths, weaknesses, and quirks. Consider how these characters interact with each other and the game world.""",
    expected_output="""Detailed character profiles, including their appearance, personality, backstory, motivations, strengths, weaknesses, and quirks. Also include a description of how they interact with each other and the game world.""",
    agent=writer
)
crew_task1 = Crew(
    agents=[writer, researcher],
    tasks=[task1],
    verbose=True
)

print("Crew1: Working on the tasks ....")

result1 = crew_task1.kickoff()

################

# Task for the game's story, incorporating results from the previous task
task2 = Task(
    description=f"""Generate a long and captivating story for the game, using the results from the following data: {result1}.
The story should have a clear beginning, middle, and end, and should be engaging enough to keep players interested throughout the game.""",
    expected_output="""A detailed game story with a clear beginning, middle, and end. The story should be engaging, with plot twists and turns that keep the player interested throughout the game.""",
    agent=writer
)


crew_task2 = Crew(
    agents=[writer, researcher],
    tasks=[task2],
    verbose=True
)
print("Crew2: Working on the tasks ....")
result2 = crew_task2.kickoff()

######################

# Task for creating detailed profiles for the game's characters, incorporating results from the previous task
task3 = Task(
    description=f"""Create detailed profiles for the game's characters, using the results from the following data: {result2}.
Each character should have a unique personality and backstory, and their actions and decisions should be consistent with their character.""",
    expected_output="""Detailed character profiles, including their appearance, personality, backstory, and motivations. Each character should feel unique, and their actions and decisions should be consistent with their character.""",
    agent=writer
)

crew_task3 = Crew(
    agents=[writer, researcher],
    tasks=[task3],
    verbose=True
)

print("Crew3 : Working on the tasks ....")
result3 = crew_task3.kickoff()
###################
# Task for creating an immersive and detailed environment for the game, incorporating results from the previous task
task4 = Task(
    description=f"""Create an immersive and detailed environment for the game, using the results from the following data: {result3}.
The environment should be a living, breathing world that reacts to the player's actions and decisions.""",
    expected_output="""A detailed description of the game's environment, including locations, weather, flora and fauna, and how they interact with the story and characters. The environment should feel alive and react to the player's actions and decisions.""",
    agent=writer
)

crew_task4 = Crew(
    agents=[writer, researcher],
    tasks=[task4],
    verbose=True
)
print("Crew4 : Working on the tasks ....")
result4 = crew_task4.kickoff()
#################################


# Task for creating interesting and challenging tasks for the game, incorporating results from the previous task
task5 = Task(
    description=f"""Create interesting and challenging tasks for the game, using the following data: {result4}.
These tasks should test the player's skills and decision-making abilities, and should be integrated into the game's story and character arcs.""",
    expected_output="""A list of challenges for the game, including their description, how they fit into the story and characters, and how they enhance the gaming experience. Each challenge should test the player's skills and decision-making abilities.""",
    agent=writer
)

crew_task5 = Crew(
    agents=[writer, researcher],
    tasks=[task5],
    verbose=True
)
print("Crew5: Working on the tasks ....")
result5 = crew_task5.kickoff()
#########################
# Task for combining all the results into a single story, incorporating results from the previous tasks
task6 = Task(
    description=f"""Combine the characters, game's story, character profiles, environment, and game's challenges into a single cohesive and engaging game story, using the following data:

Characters: {result1}
The characters are the heart of the game, each with their own unique personalities, backgrounds, strengths, weaknesses, and quirks. They interact dynamically with each other and the game world, shaping the narrative and player experience.

Game's Story: {result2}
Craft a compelling narrative arc that guides players through a rich and immersive world. From the opening scene to the final climax, the story should captivate players, presenting them with challenges, dilemmas, and moments of triumph.

Character Profiles: {result3}
Delve deeper into the backgrounds and motivations of each character. Create detailed profiles that flesh out their personalities, relationships, and personal journeys. Ensure that their actions and decisions align with their established traits.

Environment: {result4}
Design a vibrant and dynamic game world that serves as the backdrop for the adventure. Describe lush landscapes, bustling cities, eerie dungeons, and everything in between. Ensure that the environment reacts to player choices and influences the unfolding story.

Game's Challenges: {result5}
Develop engaging tasks and obstacles that test the player's skills and decision-making abilities. From puzzles and combat encounters to moral dilemmas and exploration challenges, each obstacle should enhance the narrative and immerse players further into the game world.

""",
    expected_output="""A single cohesive and engaging game story that seamlessly integrates the characters, game's story, character profiles, environment, and game's challenges. The story should be engaging, immersive, and provide a compelling gaming experience for the players.""",
    agent=writer
)


crew_task6 = Crew(
    agents=[writer, researcher],
    tasks=[task6],
    verbose=True
)

print("Crew6: Working on the tasks ....")

result6 = crew_task6.kickoff()

Crew1: Working on the tasks ....
 [DEBUG]: == Working Agent: Writer
 [INFO]: == Starting Task: Create unique and interesting characters for the game, each with their own strengths, weaknesses, and quirks. Consider how these characters interact with each other and the game world.


> Entering new CrewAgentExecutor chain...
Thought: I should ask my coworker to help me with this task
Action: Ask question to co-worker
Action Input: {
    "coworker": "Researcher",
    "question": "Can you help me create unique and interesting characters for the game, each with their own strengths, weaknesses, and quirks?",
    "context": "I'm working on creating the characters for the game, and I want to make sure that they are all unique and interesting. I'm looking for characters that have their own strengths, weaknesses, and quirks, and that interact with each other and the game world in interesting ways."
}

> Entering new CrewAgentExecutor chain...
**Thought:** I now can give a great answer.

**Final A

Thought: I now know the final answer
Final Answer: **Character Profiles:**

**Character 1: Anya**

* **Appearance:** Tall and athletic, with long, flowing brown hair and piercing blue eyes.
* **Personality:** Strong-willed and independent, with a sharp wit and a compassionate heart.
* **Backstory:** Grew up in a remote village, where she honed her survival skills and developed a deep connection to nature.
* **Motivations:** To protect her loved ones and fight for justice.
* **Strengths:** Skilled archer, adept at tracking and wilderness survival.
* **Weaknesses:** Can be impulsive and headstrong at times.
* **Quirks:** Has a habit of humming while she works, and loves to collect wildflowers.

**Character 2: Ethan**

* **Appearance:** Lean and agile, with short, dark hair and piercing green eyes.
* **Personality:** Intelligent and resourceful, with a mischievous sense of humor.
* **Backstory:** Born into a wealthy family, but left to fend for himself on the streets at a young age.
* **M

Thought: I now know the final answer
Final Answer: **Beginning:**

In the ancient realm of Eldoria, where shadows dance and secrets whisper, a group of unlikely heroes emerges. Anya, the skilled archer and tracker; Ethan, the cunning thief and hacker; Maya, the wise healer and herbalist; and Darius, the loyal and protective warrior, are drawn together by fate.

As darkness descends upon the land, an ancient evil stirs. The once-peaceful villages are ravaged by monstrous creatures, and the people live in fear. The heroes, driven by their own motivations and a shared desire to protect the innocent, embark on a perilous journey to confront the growing threat.

**Middle:**

Their journey takes them through treacherous forests, crumbling ruins, and hidden dungeons. Along the way, they face countless challenges that test their skills, their bonds, and their very souls. Anya's archery skills prove invaluable in hunting for food and defending against attacks. Ethan's stealth and hacking abilit

Thought: I now know the final answer
Final Answer: **Character Profiles**

**Anya**

* **Appearance:** Anya is a young woman with long, flowing brown hair and piercing blue eyes. She is tall and slender, with a strong and athletic build. She is often seen wearing a green tunic and leather boots, and she carries a bow and arrow on her back.
* **Personality:** Anya is a skilled archer and tracker. She is brave, resourceful, and independent. She is also fiercely loyal to her friends and family, and she is always willing to fight for what she believes in.
* **Backstory:** Anya was born into a family of hunters in the remote village of Willow Creek. She grew up learning the skills of archery and tracking, and she quickly became one of the best hunters in the village. When she was 16, her village was attacked by a group of bandits. Anya fought bravely, but she was outnumbered and forced to flee. She vowed to return and avenge her family, and she has been traveling the land ever since, honing

Thought: I now know the final answer
Final Answer: The game's environment is a vast and varied world, with a variety of different locations, weather conditions, flora, and fauna. The environment is responsive to the player's actions and decisions, and it changes and evolves as the player progresses through the game.

The game's locations include forests, mountains, deserts, and swamps. Each location has its own unique look and feel, and it is home to a variety of different plants and animals. The weather conditions in the game can also change, and they can have a significant impact on the gameplay. For example, rain can make it difficult to see, and it can also make the terrain more slippery.

The flora and fauna in the game are also responsive to the player's actions and decisions. For example, if the player cuts down a tree, the other trees in the area may start to grow faster. If the player kills an animal, the other animals in the area may become more aggressive.

The environment i

Thought: I now can give a great answer
Final Answer: **Challenge 1:**

* **Description:** The player must navigate a treacherous mountain pass. The pass is narrow and winding, with sheer cliffs on either side. The weather is stormy, with rain and high winds.
* **How it fits into the story and characters:** The player is on a quest to reach a distant land. The mountain pass is the only way to get there, but it is also very dangerous. The player must use their skills and decision-making abilities to navigate the pass safely.
* **How it enhances the gaming experience:** This challenge tests the player's skills and decision-making abilities. It also provides the player with a sense of accomplishment when they complete it.

**Challenge 2:**

* **Description:** The player must make a difficult moral choice. The choice will affect the outcome of the game.
* **How it fits into the story and characters:** The player is faced with a difficult moral dilemma. They must choose between two different

In [37]:
result6

"**Game Story:**\n\n**Beginning:**\n\nIn the realm of Eldoria, where ancient ruins whisper secrets and danger lurks, a group of unlikely heroes emerges. Anya, the skilled archer and tracker; Ethan, the cunning thief and hacker; Maya, the wise healer and herbalist; and Darius, the loyal and protective warrior, are drawn together by fate.\n\nAs darkness descends upon the land, an ancient evil stirs. The once-peaceful villages are ravaged by monstrous creatures, and the people live in fear. The heroes, driven by their own motivations and a shared desire to protect the innocent, embark on a perilous journey to confront the growing threat.\n\n**Middle:**\n\nTheir journey takes them through treacherous forests, crumbling ruins, and hidden dungeons. Along the way, they face countless challenges that test their skills, their bonds, and their very souls. Anya's archery skills prove invaluable in hunting for food and defending against attacks. Ethan's stealth and hacking abilities allow them to 